In [22]:
import sys
import pandas as pd
import numpy as np

sys.path.append("../")
import biobss
import matplotlib.pyplot as plt
import neurokit2 as nk
from plotly_resampler import register_plotly_resampler

In [23]:
import importlib
import sys
import pandas as pd
import numpy as np

sys.path.append("../")
import biobss
import os
os.getcwd()
import neurokit2 as nk
sample_data = biobss.utils.load_sample_data("ECG")

In [24]:
sr=256

In [25]:
sample=np.array(sample_data[0]).flatten()

In [26]:
clean=biobss.pipeline.Bio_Process(
    process_method=biobss.ecgtools.filter_ecg,process_name="clean",argmap={"sampling_rate":"sampling_rate"},method="pantompkins")

find_peaks = biobss.pipeline.Bio_Process(process_method=biobss.ecgtools.ecg_peaks,
                                         process_name="rpeaks",returns_event=True,event_args={"event_name":"ECG_Peaks"})


pipeline = biobss.pipeline.Bio_Pipeline(windowed_process=True,window_size=5,step_size=1)

pipeline.process_queue.add_process(clean,input_signals="ECG_Raw",output_signals="ECG_Clean")
pipeline.process_queue.add_process(find_peaks,input_signals="ECG_Clean",output_signals="ECG_Peaks")

channel=biobss.pipeline.Bio_Channel(signal=sample,name="ECG_Raw",sampling_rate=sr)
pipeline.set_input(channel)


In [27]:
pipeline

Bio_Pipeline:
	Preprocessors: Process list:

	Processors: Process list:
	1: clean(ECG_Raw) -> ECG_Clean
	2: rpeaks(ECG_Clean) -> ECG_Peaks

	Postprocessors: Process list:

	Window Size(Seconds): 5
	Step Size: 1

In [28]:
# Create feature extraction steps

# Extract time domain features. input_signals dictionary contains feature prefixes as keys and input signals as values
# For example rms of EDA_Tonic is extracted as Tonic_rms in this case
Rpeak_features = biobss.pipeline.Feature(name="time_features", function=biobss.ecgtools.from_Rpeaks, sampling_rate=sr)
stat_features = biobss.pipeline.Feature(name="stat_features", function=biobss.edatools.get_stat_features)

In [29]:
pipeline.run_pipeline()

In [30]:
pipeline.add_feature_step(Rpeak_features,input_signals=['ECG_Raw','Events_ECG_Clean'])
pipeline.add_feature_step(stat_features,input_signals=['ECG_Clean'],prefix="ECG")

In [31]:
pipeline.extract_features()

In [11]:
pipeline.features

,ecg_a_R,ecg_RR0,ecg_RR1,ecg_RR2,ecg_RRm,ecg_RR_0_1,ecg_RR_2_1,ecg_RR_m_1,ECG_mean,ECG_std,ECG_max,ECG_min,ECG_range,"(ECG_kurtosis,)","(ECG_skew,)","(ECG_momentum,)"
0,"[0.093, 0.6484375, 0.63671875, -4.5390625, -1....","[0.372, 0.63671875, -4.5390625, 0.6484375, -1....","[0.454, -4.5390625, 0.6484375, 0.63671875, -1....","[0.125, 0.6484375, 0.63671875, 0.640625, 0.641...","[0.249, 0.63671875, 0.640625, 0.671875, 0.6497...","[0.134, 0.640625, 0.671875, 0.65625, 0.65625, ...","[0.378, 0.671875, 0.65625, 0.6484375, 0.658854...","[0.389, 0.65625, 0.6484375, 0.63671875, 0.6471...","[0.00010641522711905394, nan, nan, nan, nan, n...","[nan, 0.05251431542350286, nan, nan, nan, nan,...","[nan, nan, 0.22143246887919948, nan, nan, nan,...","[nan, nan, nan, -0.13331397674247158, nan, nan...","[nan, nan, nan, nan, 0.35474644562167107, nan,...","[nan, nan, nan, nan, nan, 0.3799448279067952, ...","[nan, nan, nan, nan, nan, 0.5650546391441744, ...","[nan, nan, nan, nan, nan, 0.4293241005145907, ..."
0,"[-0.068, 0.63671875, 0.640625, -4.20703125, -0...","[0.454, 0.640625, -4.20703125, 0.3125, -1.0846...","[0.125, -4.20703125, 0.3125, 0.640625, -1.0846...","[0.249, 0.3125, 0.640625, 0.671875, 0.54166666...","[0.134, 0.640625, 0.671875, 0.65625, 0.65625, ...","[0.378, 0.671875, 0.65625, 0.6484375, 0.658854...","[0.389, 0.65625, 0.6484375, 0.63671875, 0.6471...","[0.222, 0.6484375, 0.63671875, 0.640625, 0.641...","[0.00010641522711905394, nan, nan, nan, nan, n...","[nan, 0.05251431542350286, nan, nan, nan, nan,...","[nan, nan, 0.22143246887919948, nan, nan, nan,...","[nan, nan, nan, -0.13331397674247158, nan, nan...","[nan, nan, nan, nan, 0.35474644562167107, nan,...","[nan, nan, nan, nan, nan, 0.3799448279067952, ...","[nan, nan, nan, nan, nan, 0.5650546391441744, ...","[nan, nan, nan, nan, nan, 0.4293241005145907, ..."
0,"[-0.272, 0.64453125, 0.65625, -4.515625, -1.07...","[-0.064, 0.65625, -4.515625, 0.5625, -1.098958...","[0.134, -4.515625, 0.5625, 0.7265625, -1.07552...","[0.378, 0.5625, 0.7265625, 0.6484375, 0.645833...","[0.389, 0.7265625, 0.6484375, 0.63671875, 0.67...","[0.222, 0.6484375, 0.63671875, 0.640625, 0.641...","[0.175, 0.63671875, 0.640625, 0.64453125, 0.64...","[0.436, 0.640625, 0.64453125, 0.65625, 0.64713...","[0.00010641522711905394, nan, nan, nan, nan, n...","[nan, 0.05251431542350286, nan, nan, nan, nan,...","[nan, nan, 0.22143246887919948, nan, nan, nan,...","[nan, nan, nan, -0.13331397674247158, nan, nan...","[nan, nan, nan, nan, 0.35474644562167107, nan,...","[nan, nan, nan, nan, nan, 0.3799448279067952, ...","[nan, nan, nan, nan, nan, 0.5650546391441744, ...","[nan, nan, nan, nan, nan, 0.4293241005145907, ..."
0,"[-0.034, 0.65625, 0.6328125, -4.140625, -0.950...","[-0.217, 0.6328125, -4.140625, 0.32421875, -1....","[-0.063, -4.140625, 0.32421875, 0.53515625, -1...","[0.389, 0.32421875, 0.53515625, 0.70703125, 0....","[0.222, 0.53515625, 0.70703125, 0.640625, 0.62...","[0.175, 0.70703125, 0.640625, 0.64453125, 0.66...","[0.436, 0.640625, 0.64453125, 0.65625, 0.64713...","[0.343, 0.64453125, 0.65625, 0.6328125, 0.6445...","[0.00010641522711905394, nan, nan, nan, nan, n...","[nan, 0.05251431542350286, nan, nan, nan, nan,...","[nan, nan, 0.22143246887919948, nan, nan, nan,...","[nan, nan, nan, -0.13331397674247158, nan, nan...","[nan, nan, nan, nan, 0.35474644562167107, nan,...","[nan, nan, nan, nan, nan, 0.3799448279067952, ...","[nan, nan, nan, nan, nan, 0.5650546391441744, ...","[nan, nan, nan, nan, nan, 0.4293241005145907, ..."
0,"[-0.137, 0.640625, 0.6484375, -4.45703125, -1....","[-0.077, 0.6484375, -4.45703125, 0.51953125, -...","[0.222, -4.45703125, 0.51953125, 0.71484375, -...","[0.175, 0.51953125, 0.71484375, 0.64453125, 0....","[0.436, 0.71484375, 0.64453125, 0.65625, 0.671...","[0.343, 0.64453125, 0.65625, 0.6328125, 0.6445...","[0.15, 0.65625, 0.6328125, 0.640625, 0.6432291...","[0.08, 0.6328125, 0.640625, 0.6484375, 0.64062...","[0.00010641522711905394, nan, nan, nan, nan, n...","[nan, 0.05251431542350286,